In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
     |████████████████████████████████| 204.2MB 59kB/s 
     |████████████████████████████████| 204kB 41.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=584f6722d45d12d7ce81bbdae941aeab84e474508ea4068c0102bd926d9bf20f
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
# import the DataFrame
from pyspark import SparkFiles
url="https://myfirstbucket100.s3.us-east-2.amazonaws.com/mbti.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("mbti.csv"), sep=",", header=True)
# Show DataFrame
df.show(truncate=False)

+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Model 1 : NaiveBayes classification based on 4 classes without splitting the posts

In [ ]:
import pandas as pd
pdf=df.toPandas()

Preprocess
We need to split the posts on the ||| string and create a new row with the same type

In [ ]:
import numpy as np
import re
import string
# outfilename = 'mbti_preprocessed.csv'
newrows = []

def filter_text(post):
    """Decide whether or not we want to use the post."""
    # should remove link only posts here
    return len(post) > 0
    
reg_punc = re.compile('[%s]' % re.escape(string.punctuation))
def preprocess_text(post):
    """Remove any junk we don't want to use in the post."""
    
    # Remove links
    post = re.sub(r'http\S+', '', post, flags=re.MULTILINE)
    
    # All lowercase
    post  = post.lower()
    
    # Remove puncutation
    post = reg_punc.sub('', post)

    return post

def create_new_rows(row):
    posts = row['posts']
    rows = []
    
    # for p in posts:
    p = preprocess_text(posts)
    rows.append({'type': row['type'], 'post': p})
    return rows

for index, row in pdf.iterrows():
  newrows += create_new_rows(row)
    
pdf = pd.DataFrame(newrows)
# unique = pdf.groupby('type').nunique()
# print('{} rows'.format(pdf.shape[0]))

# Preview the data
pdf.head(10)

,type,post
0,INFJ,and intj moments sportscenter not top ten ...
1,ENTP,im finding the lack of me in these posts very ...
2,INTP,good one course to which i say i know tha...
3,INTJ,dear intp i enjoyed our conversation the oth...
4,ENTJ,youre firedthats another silly misconception t...
5,INTJ,1837 science is not perfect no scientist clai...
6,INFJ,no i cant draw on my own nails haha those were...
7,INTJ,i tend to build up a collection of things on m...
8,INFJ,im not sure thats a good question the distinct...
9,INTP,in this position where i have to actually let...


Instead of doing 16 imbalanced classification head-on, this dataset can be re-classified as 4 binary classifiers!

In [ ]:
pdf['is_E'] = pdf['type'].apply(lambda x: 1 if x[0] == 'E' else 0)
pdf['is_S'] = pdf['type'].apply(lambda x: 1 if x[1] == 'S' else 0)
pdf['is_T'] = pdf['type'].apply(lambda x: 1 if x[2] == 'T' else 0)
pdf['is_J'] = pdf['type'].apply(lambda x: 1 if x[3] == 'J' else 0)

In [ ]:
df = spark.createDataFrame(pdf)
df.show()

+----+--------------------+----+----+----+----+
|type|                post|is_E|is_S|is_T|is_J|
+----+--------------------+----+----+----+----+
|INFJ| and intj moments...|   0|   0|   0|   1|
|ENTP|im finding the la...|   1|   0|   1|   0|
|INTP|good one      cou...|   0|   0|   1|   0|
|INTJ|dear intp   i enj...|   0|   0|   1|   1|
|ENTJ|youre firedthats ...|   1|   0|   1|   1|
|INTJ|1837 science  is ...|   0|   0|   1|   1|
|INFJ|no i cant draw on...|   0|   0|   0|   1|
|INTJ|i tend to build u...|   0|   0|   1|   1|
|INFJ|im not sure thats...|   0|   0|   0|   1|
|INTP| in this position...|   0|   0|   1|   0|
|INFJ|one time my paren...|   0|   0|   0|   1|
|ENFJ| oi went through ...|   1|   0|   0|   1|
|INFJ|joe santagato  en...|   0|   0|   0|   1|
|INTJ|fair enough if th...|   0|   0|   1|   1|
|INTP|basically this   ...|   0|   0|   1|   0|
|INTP|your comment scre...|   0|   0|   1|   0|
|INFJ|some of these bot...|   0|   0|   0|   1|
|INFP|i think we do agr...|   0|   0|   

In [ ]:
labels=['is_E','is_S','is_T','is_J']

In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
df = df.withColumn('length', length(df['post']))
# df.show()
df.show()

+----+--------------------+----+----+----+----+------+
|type|                post|is_E|is_S|is_T|is_J|length|
+----+--------------------+----+----+----+----+------+
|INFJ| and intj moments...|   0|   0|   0|   1|  3097|
|ENTP|im finding the la...|   1|   0|   1|   0|  6015|
|INTP|good one      cou...|   0|   0|   1|   0|  4675|
|INTJ|dear intp   i enj...|   0|   0|   1|   1|  5736|
|ENTJ|youre firedthats ...|   1|   0|   1|   1|  5339|
|INTJ|1837 science  is ...|   0|   0|   1|   1|  7979|
|INFJ|no i cant draw on...|   0|   0|   0|   1|  7215|
|INTJ|i tend to build u...|   0|   0|   1|   1|  6484|
|INFJ|im not sure thats...|   0|   0|   0|   1|  3855|
|INTP| in this position...|   0|   0|   1|   0|  6847|
|INFJ|one time my paren...|   0|   0|   0|   1|  7798|
|ENFJ| oi went through ...|   1|   0|   0|   1|  4219|
|INFJ|joe santagato  en...|   0|   0|   0|   1|  6563|
|INTJ|fair enough if th...|   0|   0|   1|   1|  8509|
|INTP|basically this   ...|   0|   0|   1|   0|  4892|
|INTP|your

In [ ]:
types = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP', 'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
types = [x.lower() for x in types]

In [ ]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="post", outputCol="words")
tokenizer

Tokenizer_2b4f2a852c67

In [ ]:
tokenized = tokenizer.transform(df)

In [ ]:
from pyspark.ml.feature import StopWordsRemover
# Remove stop words
stopwordList = types
stopwordList.extend(StopWordsRemover().getStopWords())
stopwordList = list(set(stopwordList))#optionnal
remover=StopWordsRemover(inputCol="words", outputCol="filtered" ,stopWords=stopwordList)
newFrame = remover.transform(tokenized)
# newFrame.show()

In [ ]:
from pyspark.ml.feature import HashingTF,IDF
# Run the hashing term frequency
hashing = HashingTF(inputCol="filtered", outputCol="hashedValues")
# Transform into a DF 
hashed_df = hashing.transform(newFrame) 
# hashed_df.show()

In [ ]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="idf_token")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vectors
#idf = IDF(inputCol='hash_token', outputCol='idf_token')
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')
output = clean_up.transform(rescaledData)
output.show()

+----+--------------------+----+----+----+----+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|type|                post|is_E|is_S|is_T|is_J|length|               words|            filtered|        hashedValues|           idf_token|            features|
+----+--------------------+----+----+----+----+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|INFJ| and intj moments...|   0|   0|   0|   1|  3097|[, and, intj, mom...|[, moments, , , ,...|(262144,[619,2437...|(262144,[619,2437...|(262145,[619,2437...|
|ENTP|im finding the la...|   1|   0|   1|   0|  6015|[im, finding, the...|[im, finding, lac...|(262144,[619,991,...|(262144,[619,991,...|(262145,[619,991,...|
|INTP|good one      cou...|   0|   0|   1|   0|  4675|[good, one, , , ,...|[good, one, , , ,...|(262144,[991,1578...|(262144,[991,1578...|(262145,[991,1578...|
|INTJ|dear intp   i enj...|   0|   0|   

In [ ]:
output_e=output.withColumnRenamed('is_E','label')

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = output_e.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor_e = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results_e = predictor_e.transform(testing)
test_results_e.show(10)

+----+--------------------+-----+----+----+----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|type|                post|label|is_S|is_T|is_J|length|               words|            filtered|        hashedValues|           idf_token|            features|       rawPrediction|         probability|prediction|
+----+--------------------+-----+----+----+----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|ENFJ| dont take it ser...|    1|   0|   0|   1|  5983|[, dont, take, it...|[, dont, take, se...|(262144,[1546,198...|(262144,[1546,198...|(262145,[1546,198...|[-21456.132135554...|[1.0,1.1576692907...|       0.0|
|ENFJ| welcome to the f...|    1|   0|   0|   1|  2187|[, welcome, to, t...|[, welcome, forum...|(262144,[991,1084...|(262144,[991,1084...|(2621

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results_e)
print("Accuracy of model at predicting E-type was: %f" % acc)

Accuracy of model at predicting E-type was: 0.704149


In [ ]:
output_s=output.withColumnRenamed('is_S','label')

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = output_s.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor_s = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results_s = predictor_s.transform(testing)
test_results_s.show(5)

+----+--------------------+----+-----+----+----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|type|                post|is_E|label|is_T|is_J|length|               words|            filtered|        hashedValues|           idf_token|            features|       rawPrediction|         probability|prediction|
+----+--------------------+----+-----+----+----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|ENFJ|  sveltihel brill...|   1|    0|   0|   1|  2081|[, , sveltihel, b...|[, , sveltihel, b...|(262144,[5000,844...|(262144,[5000,844...|(262145,[5000,844...|[-9135.4643268543...|[0.99999999715167...|       0.0|
|ENFJ|ability to transf...|   1|    0|   0|   1|  6439|[ability, to, tra...|[ability, transfo...|(262144,[1141,202...|(262144,[1141,202...|(2621

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results_s)
print("Accuracy of model at predicting S-type was: %f" % acc)

Accuracy of model at predicting S-type was: 0.824022


In [ ]:
output_t=output.withColumnRenamed('is_T','label')

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = output_t.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor_t = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results_t = predictor_t.transform(testing)
test_results_t.show(5)

+----+--------------------+----+----+-----+----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|type|                post|is_E|is_S|label|is_J|length|               words|            filtered|        hashedValues|           idf_token|            features|       rawPrediction|         probability|prediction|
+----+--------------------+----+----+-----+----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|ENFJ| oi went through ...|   1|   0|    0|   1|  4219|[, oi, went, thro...|[, oi, went, brea...|(262144,[844,1546...|(262144,[844,1546...|(262145,[844,1546...|[-10469.043219193...|[0.47408557708877...|       1.0|
|ENFJ|1 i am an anime a...|   1|   0|    0|   1|  2844|[1, i, am, an, an...|[1, anime, addict...|(262144,[991,1842...|(262144,[991,1842...|(2621

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results_t)
print("Accuracy of model at predicting T-type was: %f" % acc)

Accuracy of model at predicting T-type was: 0.710096


In [ ]:
output_j=output.withColumnRenamed('is_J','label')

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = output_j.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor_j = nb.fit(training)

In [ ]:
predictor_e.save("EI_Predictor.h5")
predictor_s.save("SN_Predictor.h5")
predictor_t.save("TF_Predictor.h5")
predictor_j.save("JP_Predictor.h5")

In [ ]:
'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin*  That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This + Lack of Balance and Hand Eye Coordination.|||Real IQ test I score 127. Internet IQ tests are funny. I score 140s or higher.  Now, like the former responses of this thread I will mention that I don't believe in the IQ test. Before you banish...|||You know you're an ENTP when you vanish from a site for a year and a half, return, and find people are still commenting on your posts and liking your ideas/thoughts. You know you're an ENTP when you...|||http://img188.imageshack.us/img188/6422/6020d1f9da6944a6b71bbe6.jpg|||http://img.adultdvdtalk.com/813a0c6243814cab84c51|||I over think things sometimes. I go by the old Sherlock Holmes quote.  Perhaps, when a man has special knowledge and special powers like my  own, it rather encourages him to seek a complex...|||cheshirewolf.tumblr.com  So is I :D|||400,000+  post|||Not really; I've never thought of E/I or J/P as real functions.  I judge myself on what I use. I use Ne and Ti as my dominates. Fe for emotions and rarely Si. I also use Ni due to me strength...|||You know though. That was ingenious. After saying it I really want to try it and see what happens with me playing a first person shooter in the back while we drive around. I want to see the look on...|||out of all of them the rock paper one is the best. It makes me lol.  You guys are lucky :D I'm really high up on the tumblr system.|||So did you hear about that new first person shooter game? I've been rocking the hell out of the soundtrack on my auto sound equipment that will shake the heavens. We managed to put a couple PS3's in...|||No; The way he connected things was very Ne. Ne dominates are just as aware of their environments as Se dominates.  Example: Shawn Spencer or Patrick Jane; Both ENTPs.|||Well charlie I will be the first to admit I do get jealous like you do. I chalk it up to my 4w3 heart mixed with my dominate 7w8. 7s and 8s both like to be noticed. 4's like to be known (not the same...|||;D I'll upload the same clip with the mic away from my mouth. Than you won't hear anything.  Ninja Assassin style but with splatter.|||Tik Tok is a really great song. As long as you can mental block out the singer. I love the beat it makes me bounce.|||drop.io v1swck0  :D Mic really close to my mouth and smokin aces: assassins ball playing in the background.|||Sociable =/= extrovert; I'm an extrovert and I'm not sociable. :)|||Sherlock in the movie was an ENTP. Normally he's played as a EXTJ. In the books he's an ESTJ.  As I said. The movie looked good except for it being called sherlock holmes.|||http://i817.photobucket.com/albums/zz96/kamioo/Dirtywinch.png|||Oh, I never had fear of kissing a guy. I will kiss an animal too. So there was nothing to vanish. Just personal taste and me not liking it.  The guy I kissed didn't know me. It was one of those...|||Sounds pretty much like my area and what I'm going through right now trying to figure out which way I want to take my life. I want to do so many things. The biggest problem is that I know if I don't...|||;D I was operating under the impression that you were female. I never looked at your boxy. Okay, I help out my gay friends all the time and one of them has developed a little crush on me. I get red...|||T_T You just described me  and I'm living the worst nightmare. I'm trapped in one place with one one around. Only dull woods. If I was a serial killer this would be the perfect place but sadly I'm...|||TBH, and biased, sounds like a shadowed INFP. I think maybe he was hurt and turned ESTJ. I can tell because he has some of the typical INFP traits left over.|||*Checks list* I'm sorry. It seems that you have came at a bad time. We've already reached our quota of INFJs. However, being you're female and I like females I will make you a deal. I will kick one...|||I'm ANTP (Leaning toward E). I'm easy for both ENTPs and INTPs to identify with. :)|||I also imagine ENTP's interrogations would go a little bit like Jack's from 24 except more mechanical. Rigging up shock treatment equipment in an abandoned building out of an old car batty, jumper...|||It was a compliment :) Trust me. I'm just as psychopathic :D except I have emoticons. They're just weird ones. Like laughing when I get hurt or at people running themselves over with their lawn mower...|||http://i817.photobucket.com/albums/zz96/kamioo/Thunderstorm.pnghttp://i817.photobucket.com/albums/zz96/kamioo/Thunderstormbw.png http://i817.photobucket.com/albums/zz96/kamioo/Cosmicstorm.png|||No. It's like a theme for where I live and that is why I know it by heart.   http://www.youtube.com/watch?v=j5W73HaVQBg|||and I usual don't leave until the thing ends. But in the mean time. In between times. You work your thing. I'll work mine :D  ;D I'm the MBP; Pleasure to meet you.|||Damn, need to trust my instincts more I would have been closer I was going to say INFP.|||EXFP? Leaning toward S with the way she responded.  :D My friends, even my gay and lesbian ones, always come to me for advice.|||I bow to my entp masters ENTPs are so great. If it wasn't for ENTPs I wouldn't have been able to build what I'm building  Duck Duck  Duck  Shotgun|||What? Me? I never do that >.> <.<|||Because its hard to be sad about losing someone you like when you knew you were right and give yourself a big pat on the back because you're awesome and always correct.|||Oh, you don't have to tell me that most of them are stupid. I know this. That is why I play with them and it makes me laugh. :D As I'm going to take Neuropsychology and I have a few psychologist...|||:D I'm a Nightowl. I wake up between 6-7pm and stay awake till 10-11:30am.|||Personal opinion backed by theory would suggest that INTPs are the most socially difficult. While INTJs can be socially indifferent but they will also use social situations if the the need arises....|||Personal stocks that I have on my desktop that I've downloaded from random stock sites and stock photobuckets.|||I'll tell you when I open photoshop.  :) Glad you like it static.|||:D Thanks.|||http://i817.photobucket.com/albums/zz96/kamioo/Deathgrip.png http://i817.photobucket.com/albums/zz96/kamioo/Deathgripbw.png  Made for a friend. Several hours of work. I constructed every line by...|||:) Static: http://i817.photobucket.com/albums/zz96/kamioo/Statickitten.png  I'll have to get to your avatar later if one of my fellow teammates doesn't.|||Psychologist don't keep me around long enough to diagnosis me. I like to toy with them. What I have diagnosis myself with and had a few psychologist friends (+ a few other friends) tell me I have is...'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

# Model 2 : NaiveBayes classification based on 4 classes with prerocessing

In [ ]:
from pyspark.sql.functions import col
import pyspark.sql.functions as f
from pyspark.sql.window import Window
df.groupBy('type').count()\
  .withColumn('percentage', f.round(100*f.col('count') / f.sum('count')\
  .over(Window.partitionBy()),3)).sort(col("count"),ascending=False).show(truncate=False)

In [ ]:
import pandas as pd
pdf=df.toPandas()
pdf.to_csv('MBTI.csv')

# Sklearn Model 

In [ ]:

# import numpy as np
# import os
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.naive_bayes import MultinomialNB
# import csv
# from sklearn.metrics import classification_report
# from sklearn.model_selection import train_test_split

# X_data = []
# y_data = []

# def regex_tokenizer(doc):
#     """Return a function that split a string in sequence of tokens"""
#     return doc.split(' ')

# with open('MBTI.csv') as csvFile:
#     reader = csv.reader(csvFile)
#     x = 0
#     for line in reader:
#         if line[1] ==" ":
#             continue
#         X_data.append(line[1])
#         y_data.append(line[2])
#         x +=1;
#         if x == 20000:
#             break         

# X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2)

# vectorizer = CountVectorizer(lowercase=False, stop_words=None,  max_df=1.0, min_df=1, max_features=None, tokenizer=regex_tokenizer )

# X_train = vectorizer.fit_transform(X_train).toarray()
# X_test = vectorizer.transform(X_test).toarray()

# # print(X_train)

# y_test = np.asarray(y_test)
# y_train = np.asarray(y_train)

# # print(len(X_train))
# # print(len(y_train))

# clf = MultinomialNB()
# clf.fit(X_train,y_train)
# prediction = clf.predict(X_test)
# # print(classification_report(y_test, prediction))

#Preprocess
We need to split the posts on the ||| string and create a new row with the same type

In [ ]:
import numpy as np
import re
import string
outfilename = 'mbti_preprocessed.csv'
newrows = []

def filter_text(post):
    """Decide whether or not we want to use the post."""
    # should remove link only posts here
    return len(post) > 0
    
reg_punc = re.compile('[%s]' % re.escape(string.punctuation))
def preprocess_text(post):
    """Remove any junk we don't want to use in the post."""
    
    # Remove links
    post = re.sub(r'http\S+', '', post, flags=re.MULTILINE)
    
    # All lowercase
    post  = post.lower()
    
    # Remove puncutation
    post = reg_punc.sub('', post)

    return post

def create_new_rows(row):
    posts = row['posts'].split('|||')
    rows = []
    
    for p in posts:
        p = preprocess_text(p)
        if not filter_text(p):
            continue
        rows.append({'type': row['type'], 'post': p})
    return rows

for index, row in pdf.iterrows():
  newrows += create_new_rows(row)
    
pdf = pd.DataFrame(newrows)
unique = pdf.groupby('type').nunique()
print('{} rows'.format(pdf.shape[0]))

# Preview the data
pdf.head(10)

In [ ]:
unique.sort_values(by=['post'], ascending=False)

Save preprocessed data to csv

In [ ]:
pdf.to_csv('mbti_preprocessed.csv')
pd.read_csv('mbti_preprocessed.csv').head()

Instead of doing 16 imbalanced classification head-on, this dataset can be re-classified as 4 binary classifiers!

In [ ]:
pdf['is_E'] = pdf['type'].apply(lambda x: 1 if x[0] == 'E' else 0)
pdf['is_S'] = pdf['type'].apply(lambda x: 1 if x[1] == 'S' else 0)
pdf['is_T'] = pdf['type'].apply(lambda x: 1 if x[2] == 'T' else 0)
pdf['is_J'] = pdf['type'].apply(lambda x: 1 if x[3] == 'J' else 0)
# pdf.head()

In [ ]:
# pdf['label'] = pdf[pdf.columns[2:]].values.tolist()

In [ ]:
#pdf=pdf.drop(columns=['is_E','is_S','is_T','is_J'])
# pdf.head()

In [ ]:
# Create a Spark DataFrame from a pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

In [ ]:
# df.show()

In [ ]:
labels=['is_E','is_S','is_T','is_J']

In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
df = df.withColumn('length', length(df['post']))
# df.show()

 <!-- There are a total of 16 different combinations of MBTI types, which means we would have a 16-class classification problem to solve- -->

In [ ]:
types = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP', 'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
types = [x.lower() for x in types]


In [ ]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="post", outputCol="words")
tokenizer

In [ ]:
# Tokenize word
# Transform DataFrame
tokenized = tokenizer.transform(df)
# tokenized.show()

In [ ]:

from pyspark.ml.feature import StopWordsRemover
# Remove stop words
stopwordList = types
stopwordList.extend(StopWordsRemover().getStopWords())
stopwordList = list(set(stopwordList))#optionnal
remover=StopWordsRemover(inputCol="words", outputCol="filtered" ,stopWords=stopwordList)
newFrame = remover.transform(tokenized)
newFrame.show()


In [ ]:
# to get rid of white spaces
# from pyspark.sql.functions import regexp_replace
# newFrame.select((regexp_replace(col("filtered")[i], ", ,", "")))

In [ ]:
from pyspark.ml.feature import HashingTF,IDF
# Run the hashing term frequency
hashing = HashingTF(inputCol="filtered", outputCol="hashedValues")
# Transform into a DF 
hashed_df = hashing.transform(newFrame) 
# hashed_df.show()

In [ ]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="idf_token")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [ ]:
# Display the DataFrame
# rescaledData.show()

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vectors
#idf = IDF(inputCol='hash_token', outputCol='idf_token')
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')
output = clean_up.transform(rescaledData)
output.show()

In [ ]:
output_e=output.withColumnRenamed('is_E','label')

In [ ]:
# from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = output_e.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
# nb = RandomForestClassifier()
nb = NaiveBayes()
predictor_e = nb.fit(training)

In [ ]:
# pandas_df = output.toPandas()

In [ ]:
# Tranform the model with the testing data
test_results_e = predictor_e.transform(testing)
test_results_e.show(10)

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results_e)
print("Accuracy of model at predicting E-type was: %f" % acc)

In [ ]:
output_s=output.withColumnRenamed('is_S','label')

In [ ]:
# from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = output_s.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
# nb = RandomForestClassifier()
nb = NaiveBayes()
predictor_s = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results_s = predictor_s.transform(testing)
test_results_s.show(5)

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results_s)
print("Accuracy of model at predicting S-type was: %f" % acc)

In [ ]:
output_t=output.withColumnRenamed('is_T','label')

In [ ]:
# from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = output_t.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
# nb = RandomForestClassifier()
nb = NaiveBayes()
predictor_t = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results_t = predictor_t.transform(testing)
test_results_t.show(5)

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results_t)
print("Accuracy of model at predicting T-type was: %f" % acc)

In [ ]:
output_j=output.withColumnRenamed('is_J','label')

In [ ]:
# from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = output_j.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
# nb = RandomForestClassifier()
nb = NaiveBayes()
predictor_j = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results_j = predictor_j.transform(testing)
test_results_j.show(5)

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results_j)
print("Accuracy of model at predicting J-type was: %f" % acc)

# Saving the Model